In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch

In [2]:
from src.vlstm_v2.interface import testkernel, copykernel, mmkernelv1

INCLUDE: ['/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/TH', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/THC', '/scratch/maximilian.beck/.conda/envs/xlstmdev1/include']
/iarai/home/maximilian.beck/repos/vlstm_cuda/src

/scratch/maximilian.beck/.conda/envs/xlstmdev1/lib


Using /scratch/maximilian.beck/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /scratch/maximilian.beck/.cache/torch_extensions/py311_cu118/vlstm_v2/build.ninja...
Building extension module vlstm_v2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /scratch/maximilian.beck/.conda/envs/xlstmdev1/bin/nvcc  -ccbin /scratch/maximilian.beck/.conda/envs/xlstmdev1/bin/x86_64-conda-linux-gnu-cc -DTORCH_EXTENSION_NAME=vlstm_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/TH -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/lib/python3.11/site-packages/torch/include/THC -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/include -isystem /scratch/maximilian.beck/.conda/envs/xlstmdev1/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS_

Loading extension module vlstm_v2...


In [3]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [4]:
mat_A = torch.arange(4).reshape(2, 2).to(dtype=DTYPE, device=DEVICE)
mat_A

tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [5]:
out = testkernel(mat_A)
out

Test kernel!


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.bfloat16)

In [6]:
out = copykernel(mat_A.to(dtype=torch.float16))
out 

before kernel dispatch - float16!
rows: 2, cols: 2
blocksxy: 1-1, threads: 32-32
cidx: 0, ridx: 0, val: 0.000000
cidx: 1, ridx: 0, val: 2.000000
cidx: 0, ridx: 1, val: 1.000000
cidx: 1, ridx: 1, val: 3.000000


tensor([[0., 1.],
        [2., 3.]], device='cuda:0', dtype=torch.float16)

In [7]:
warp_size = 32
am = torch.ones((1*1*warp_size, 2*1*warp_size), device=DEVICE, dtype=DTYPE)
bm = 2 * torch.ones((2*1*warp_size, 1*1*warp_size), device=DEVICE, dtype=DTYPE)
cm = am @ bm
cm, cm.shape

(tensor([[128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.],
         ...,
         [128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.],
         [128., 128., 128.,  ..., 128., 128., 128.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.Size([32, 32]))

In [8]:
dt = torch.bfloat16
am = am.to(dtype=dt)
bm = bm.to(dtype=dt)
mmkernelv1(mat_A=am, mat_B=bm)

before kernel dispatch - bfloat16!
m: 32, n: 32, k: 64
blocksxy: 4-4, threads: 8-8
In Kernel: m: 32, n: 32, k: 64
In Kernel: gdim.x: 4, gdim.y: 4, bdim.x: 8, bdim.y: 8


tensor([[128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.],
        ...,
        [128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.],
        [128., 128., 128.,  ..., 128., 128., 128.]], device='cuda:0',
       dtype=torch.bfloat16)

In [48]:
S = 8 # sequence length
DH = 8 # hidden size

In [49]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape

(torch.Size([16, 8]), torch.Size([8, 8]))

In [50]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [604., 604., 604., 604., 604., 604., 604., 604.],
         [668., 668., 668., 668., 668., 668., 668., 668.],
         [732., 732., 732., 732., 732., 732., 732., 732.],
         [796., 796., 796., 796., 796., 796., 796., 796.],
         [860., 860., 860., 860., 860., 860., 860., 860.],
         [924., 924., 924., 924., 924., 924., 924., 924.],
         [988., 988., 988., 988., 988., 988., 988., 988.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.

In [51]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [52]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

before kernel dispatch - bfloat16!
m: 16, n: 8, k: 8
blocksxy: 1-2, threads: 8-8
In Kernel: m: 16, n: 8, k: 8
In Kernel: gdim.x: 1, gdim.y: 2, bdim.x: 8, bdim.y: 8


(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [608., 608., 608., 608., 608., 608., 608., 608.],
         [672., 672., 672., 672., 672., 672., 672., 672.],
         [736., 736., 736., 736., 736., 736., 736., 736.],
         [800., 800., 800., 800., 800., 800., 800., 800.],
         [864., 864., 864., 864., 864., 864., 864., 864.],
         [928., 928., 928., 928., 928., 928., 928., 928.],
         [992., 992., 992., 992., 992., 992., 992., 992.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.

In [14]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [15]:
# mat @ mat.T @ mat

### pytorch version

In [16]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [17]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))